In [1]:
    #IMPORTING DEPENDENCIES

In [2]:
pip install Keras-Preprocessing


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 42.6/42.6 kB 1.8 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [3]:
import re
import pandas as pd
import numpy as np
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.text import Tokenizer
from keras.utils import pad_sequences
import keras
from sklearn.metrics import classification_report
from sklearn.metrics import accuracy_score
import math
import nltk

2024-05-08 15:36:55.018443: E external/local_xla/xla/stream_executor/cuda/cuda_dnn.cc:9261] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2024-05-08 15:36:55.018542: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:607] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-05-08 15:36:55.157952: E external/local_xla/xla/stream_executor/cuda/cuda_blas.cc:1515] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered


In [4]:
r = pd.read_csv('/kaggle/input/imdb-dataset-of-50k-movie-reviews/IMDB Dataset.csv')
r

,review,sentiment
0,One of the other reviewers has mentioned that ...,positive
1,A wonderful little production. <br /><br />The...,positive
2,I thought this was a wonderful way to spend ti...,positive
3,Basically there's a family where a little boy ...,negative
4,"Petter Mattei's ""Love in the Time of Money"" is...",positive
...,...,...
49995,I thought this movie did a down right good job...,positive
49996,"Bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,I am a Catholic taught in parochial elementary...,negative
49998,I'm going to have to disagree with the previou...,negative


In [5]:
#DATA PREPROCESSING

In [6]:
r['review'] = r['review'].str.replace(r'<[^<>]*>', '')
r['review'] = r['review'].replace(to_replace=r'^https?:\/\/.*[\r\n]*',value='')
r['review'] = r['review'].str.lower()

In [7]:
d = pd.DataFrame({'strings': ['a#bc1!', 'a(b$c']})

d.strings.str.replace('[^a-zA-Z0-9]', '')

0    a#bc1!
1     a(b$c
Name: strings, dtype: object

In [8]:
r

,review,sentiment
0,one of the other reviewers has mentioned that ...,positive
1,a wonderful little production. <br /><br />the...,positive
2,i thought this was a wonderful way to spend ti...,positive
3,basically there's a family where a little boy ...,negative
4,"petter mattei's ""love in the time of money"" is...",positive
...,...,...
49995,i thought this movie did a down right good job...,positive
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative
49997,i am a catholic taught in parochial elementary...,negative
49998,i'm going to have to disagree with the previou...,negative


In [9]:
#RECURRING WORDS

In [10]:

# for i in range(len(r['review'])):
#     pd.Series(" ".join(r['review']).split()).value_counts()>100

In [11]:
rem_stopwords = ['it.', 'could', 'made', '-', 'get', 'also', 'would', 'get', 'seen', 'go', 'two', 'one']

In [12]:
#REMOVING STOPWORDS

In [13]:
nltk.download('stopwords')
from nltk.corpus import stopwords
stop_words = set(stopwords.words('english'))
r['review'] = r['review'].apply(lambda x: " ".join([word for word in x.split() if word not in stop_words]))
r['review'] = r['review'].apply(lambda x: " ".join([word for word in x.split() if word not in rem_stopwords]))

[nltk_data] Downloading package stopwords to /usr/share/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [14]:
#TOKENIZATION

In [15]:
# from pandarallel import pandarallel

# pandarallel.initialize(progress_bar=True)
# r['tokenized'] = r['review'].parallel_apply(lambda x: nltk.word_tokenize(x))

In [16]:
# r['review'].apply(lambda x: len(x.split()))

In [19]:
!unzip /usr/share/nltk_data/corpora/wordnet.zip -d /usr/share/nltk_data/corpora

Archive:  /usr/share/nltk_data/corpora/wordnet.zip
   creating: /usr/share/nltk_data/corpora/wordnet/
  inflating: /usr/share/nltk_data/corpora/wordnet/lexnames  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adv  
  inflating: /usr/share/nltk_data/corpora/wordnet/adv.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.verb  
  inflating: /usr/share/nltk_data/corpora/wordnet/cntlist.rev  
  inflating: /usr/share/nltk_data/corpora/wordnet/data.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.adj  
  inflating: /usr/share/nltk_data/corpora/wordnet/LICENSE  
  inflating: /usr/share/nltk_data/corpora/wordnet/citation.bib  
  inflating: /usr/share/nltk_data/corpora/wordnet/noun.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/verb.exc  
  inflating: /usr/share/nltk_data/corpora/wordnet/README  
  inflating: /usr/share/nltk_data/corpora/wordnet/index.sense  
  inflating: /usr/share/nltk_data

In [22]:
from textblob import Word
r['lemmatized'] = r['review'].apply(lambda x: " ".join(Word(word).lemmatize() for word in str(x).split()))

In [23]:
recuuring_words = pd.Series(" ".join(r['review']).split()).value_counts()

In [24]:
(recuuring_words>10).value_counts()     #choosing vocab_size

count
False    350856
True      39899
Name: count, dtype: int64

In [25]:
r

,review,sentiment,lemmatized
0,reviewers mentioned watching 1 oz episode hook...,positive,reviewer mentioned watching 1 oz episode hooke...
1,wonderful little production. <br /><br />the f...,positive,wonderful little production. <br /><br />the f...
2,thought wonderful way spend time hot summer we...,positive,thought wonderful way spend time hot summer we...
3,basically there's family little boy (jake) thi...,negative,basically there's family little boy (jake) thi...
4,"petter mattei's ""love time money"" visually stu...",positive,"petter mattei's ""love time money"" visually stu..."
...,...,...,...
49995,thought movie right good job. creative origina...,positive,thought movie right good job. creative origina...
49996,"bad plot, bad dialogue, bad acting, idiotic di...",negative,"bad plot, bad dialogue, bad acting, idiotic di..."
49997,catholic taught parochial elementary schools n...,negative,catholic taught parochial elementary school nu...
49998,i'm going disagree previous comment side malti...,negative,i'm going disagree previous comment side malti...


In [26]:
# pos = 0
# for i in range(r.shape[0]):
#                if r.iloc[i]['sentiment'] == 'positive':
#                     pos = pos+1
# neg = r.shape[0]-pos
# posper = pos/r.shape[0]*100
# negper = neg/r.shape[0]*100
# print(posper)
# print(negper)

In [27]:
#ENCODING

In [28]:
reviews = r['lemmatized'].values
labels = r['sentiment'].values
encoder = LabelEncoder()
encoded_labels = encoder.fit_transform(labels)    #converting positive  to 1 and negative to 0

In [29]:
#SPLITTING TRAINING AND TESTING DATA

In [30]:
train_sentences, test_sentences, train_labels, test_labels = train_test_split(reviews, encoded_labels, stratify= encoded_labels)

In [31]:
#HYPERPARAMETERS OF MODEL

In [32]:
vocab_size = 10000
tok = ''
embedding_dim = 100
max_length = 300
padding_type = 'post' 
trunc_type = 'post'

In [33]:
#TOKENIZE SENTENCES

In [34]:
tokenize = Tokenizer(num_words = vocab_size, oov_token =tok)
tokenize.fit_on_texts(train_sentences)
word_index = tokenize.word_index

In [35]:
#CONVERTING TRAIN DATASET TO SEQUENCES AND PAD SEQUENCES

In [36]:
train_sequences = tokenize.texts_to_sequences(train_sentences)
train_padding = pad_sequences(train_sequences, padding = 'post', maxlen = max_length)

In [37]:
sequence_length = max(len(sentence.split()) for sentence in train_sentences)


In [38]:
print(sequence_length)

1436


In [39]:
#CONVERTING TEST DATASET TO SEQUENCES AND PAD SEQUENCES

In [40]:
test_sequences = tokenize.texts_to_sequences(test_sentences)
test_padding = pad_sequences(test_sequences, padding = 'post', maxlen = max_length)

In [42]:
#BUILDING MODEL
model = keras.Sequential([
    keras.layers.Embedding(vocab_size, embedding_dim),
    keras.layers.Bidirectional(keras.layers.LSTM(64)),
    keras.layers.Dense(24, activation='relu'),
    keras.layers.Dense(1, activation='sigmoid')
])
model.build((None, sequence_length))

In [43]:
#Initialisation

In [44]:
#COMPILING

In [45]:
model.compile(loss = 'binary_crossentropy',
              optimizer  = 'adam',
              metrics = ['accuracy'])

In [46]:
model.summary()

Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ embedding (Embedding)           │ (None, 1436, 100)      │     1,000,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ bidirectional (Bidirectional)   │ (None, 128)            │        84,480 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 24)             │         3,096 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 1)              │            25 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 1,087,601 (4.15 MB)

 Trainable params: 1,087,601 (4.15 MB)

 Non-trainable params: 0 (0.00 B)

In [47]:
#TRAINING

In [48]:
n_epochs= 20
history = model.fit(train_padding, train_labels,
                   epochs=n_epochs, verbose= 1,
                   validation_split = 0.1) 

Epoch 1/20
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 36s 30ms/step - accuracy: 0.7490 - loss: 0.4955 - val_accuracy: 0.7904 - val_loss: 0.4650
Epoch 2/20
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 32s 30ms/step - accuracy: 0.8863 - loss: 0.2953 - val_accuracy: 0.8851 - val_loss: 0.2921
Epoch 3/20
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 32s 30ms/step - accuracy: 0.9363 - loss: 0.1813 - val_accuracy: 0.8904 - val_loss: 0.2817
Epoch 4/20
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 32s 31ms/step - accuracy: 0.9518 - loss: 0.1428 - val_accuracy: 0.8899 - val_loss: 0.3188
Epoch 5/20
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 32s 30ms/step - accuracy: 0.9602 - loss: 0.1177 - val_accuracy: 0.8928 - val_loss: 0.3474
Epoch 6/20
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 32s 30ms/step - accuracy: 0.9725 - loss: 0.0828 - val_accuracy: 0.8811 - val_loss: 0.3630
Epoch 7/20
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 41s 31ms/step - accuracy: 0.9767 - loss: 0.0720 - val_accuracy: 0.8888 - val_loss: 0.4698
Epoch 8/20
1055/1055 ━━━━━━━━━━━━━━━━━━━━ 32s 30ms/step - accuracy: 0.9885 -

In [49]:
prediction = model.predict(test_padding)


391/391 ━━━━━━━━━━━━━━━━━━━━ 5s 11ms/step


In [50]:
#LABELNG BASED ON PROBABILITY

In [51]:
p_labels = []
for i in prediction:
    if i > 0.5:
        p_labels.append(1)
    elif i==0.5:
        p_labels.append(2)
    else:
        p_labels.append(0)
print("Accuracy: ", accuracy_score(test_labels,p_labels))

Accuracy:  0.862


In [52]:
sentence = ["The movie was best",
           "The plot of movie was terrible",
           "The actors did a very good job",
           'beautiful movie. loved it',
           "it was boring",
           "VFX were a spectacle to see",
           "i loved the movie",
           " It stinks"]
sequences = tokenize.texts_to_sequences(sentence)
padding = pad_sequences(sequences, padding = 'post', maxlen = max_length)
prediction = model.predict(padding)
p_labels = []
for i in prediction:
    if i > 0.5:
        p_labels.append(1)
    elif i==0.5:
        p_labels.append(2)
    else:
        p_labels.append(0)
for i in range (len(sentence)):
    if p_labels[i]==1:
        s= 'Positive'
    elif p_labels[i]==2:
        s='Neutral'
    else:
        s='Negative'
    print("Sentiment"+str(i)+" is: ",s)


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step
Sentiment0 is:  Positive
Sentiment1 is:  Negative
Sentiment2 is:  Positive
Sentiment3 is:  Positive
Sentiment4 is:  Negative
Sentiment5 is:  Positive
Sentiment6 is:  Positive
Sentiment7 is:  Negative


In [54]:
model.save('senty.keras')